In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import pyaudio
import wave
import threading
import time
from typing import List, Tuple, Dict
import queue
from dataclasses import dataclass

# Audio synthesis constants
SAMPLE_RATE = 44100
CHUNK_SIZE = 1024
NYQUIST = SAMPLE_RATE / 2

@dataclass
class HandData:
    landmarks: List
    is_right: bool
    bbox: Tuple[int, int, int, int]

class AdvancedFMSynth:
    """Real-time FM polyphonic synthesizer with effects."""
    
    def __init__(self):
        self.audio_queue = queue.Queue()
        self.is_recording = False
        self.recording_buffer = []
        
        # FM synthesis parameters
        self.voices = {}
        self.carrier_freqs = {}
        self.modulator_freqs = {}
        self.volume_envelope = {}
        
        # Effects
        self.reverb_time = 0.3
        self.delay_time = 0.2
        self.distortion = 0.0
        
    def fm_synth_sample(self, carrier_freq: float, mod_freq: float, 
                       mod_index: float = 5.0, duration: float = 1/60) -> np.ndarray:
        """Generate one frame of FM synthesis."""
        t = np.linspace(0, duration, int(SAMPLE_RATE * duration), False)
        
        # Carrier + modulator
        modulator = np.sin(2 * np.pi * mod_freq * t)
        carrier = np.sin(2 * np.pi * carrier_freq * t + mod_index * modulator)
        
        # ADSR envelope (simple decay)
        envelope = np.exp(-5 * t)
        return carrier * envelope
    
    def apply_effects(self, samples: np.ndarray) -> np.ndarray:
        """Reverb + delay + distortion."""
        output = samples.copy().astype(np.float32)
        
        # Distortion
        if self.distortion > 0:
            output = np.tanh(output * (1 + self.distortion))
        
        # Simple delay
        delay_samples = int(self.delay_time * SAMPLE_RATE)
        if len(output) > delay_samples:
            output[delay_samples:] += 0.3 * output[:-delay_samples]
        
        # Reverb (comb filter)
        reverb_samples = int(self.reverb_time * SAMPLE_RATE)
        if len(output) > reverb_samples:
            output[reverb_samples:] += 0.2 * output[:-reverb_samples]
        
        return np.clip(output, -1.0, 1.0)
    
    def generate_audio(self):
        """Audio callback - generates continuous stream."""
        p = pyaudio.PyAudio()
        stream = p.open(format=pyaudio.paFloat32,
                       channels=1,
                       rate=SAMPLE_RATE,
                       output=True,
                       frames_per_buffer=CHUNK_SIZE)
        
        while True:
            chunk = np.zeros(CHUNK_SIZE, dtype=np.float32)
            
            # Mix active voices
            for voice_id, (freq, mod_freq) in self.voices.items():
                voice_samples = self.fm_synth_sample(freq, mod_freq)
                chunk[:len(voice_samples)] += voice_samples[:CHUNK_SIZE]
            
            # Apply effects and normalize
            chunk = self.apply_effects(chunk)
            chunk /= max(1, len(self.voices))
            
            # Record if active
            if self.is_recording:
                self.recording_buffer.extend(chunk)
            
            stream.write(chunk.astype(np.float32).tobytes())
        
        stream.stop_stream()
        stream.close()
        p.terminate()

class GestureOrchestra:
    """Advanced gesture-to-music mapping."""
    
    def __init__(self):
        self.synth = AdvancedFMSynth()
        self.chord_progressions = {
            'pop': ['C', 'G', 'Am', 'F'],
            'punk': ['G', 'D', 'Em', 'C'], 
            'jazz': ['Cmaj7', 'Dm7', 'G7', 'Cmaj7'],
            'sad': ['Am', 'F', 'C', 'G']
        }
        self.current_progression = 0
        self.progression_step = 0
        
    def fingers_up(self, landmarks: List) -> List[bool]:
        """Advanced finger detection with hysteresis."""
        fingers = []
        # Thumb (x-axis)
        fingers.append(landmarks[4].x < landmarks[3].x * 0.97)
        # Other fingers (y-axis with smoothing)
        for tip, pip in [(8,6), (12,10), (16,14), (20,18)]:
            fingers.append(landmarks[tip].y < landmarks[pip].y * 0.98)
        return fingers
    
    def hand_position(self, landmarks: List, frame_shape: Tuple) -> Dict:
        """3D-aware hand position and gesture parameters."""
        h, w = frame_shape[:2]
        wrist = landmarks[0]
        
        # Normalized position
        pos = {
            'x': wrist.x,
            'y': wrist.y,
            'z': wrist.z,  # Depth
            'pitch': (wrist.y - 0.5) * 12,  # Height = pitch bend
            'spread': self._finger_spread(landmarks)  # Vibrato
        }
        return pos
    
    def _finger_spread(self, landmarks: List) -> float:
        """Finger spread for vibrato."""
        tips = [landmarks[i] for i in [8,12,16,20]]
        distances = []
        for i in range(4):
            for j in range(i+1, 4):
                dist = np.sqrt((tips[i].x-tips[j].x)**2 + (tips[i].y-tips[j].y)**2)
                distances.append(dist)
        return np.mean(distances)
    
    def recognize_gesture(self, fingers: List[bool], hand_pos: Dict, is_right: bool) -> Dict:
        """Advanced gesture recognition with context."""
        fingers_str = ''.join('1' if f else '0' for f in fingers)
        
        gestures = {
            '00000': 'stop',  # Fist
            '10000': 'record_toggle' if is_right else 'progression_next',  # Thumb
            '00001': 'fx_cycle',  # Pinky
            '11111': 'open_chord',  # Palm
            '00100': 'bass_note',  # Middle finger
            '10101': 'arpeggio',  # Multiple fingers
        }
        
        gesture = gestures.get(fingers_str, 'sustain')
        return {
            'type': gesture,
            'pitch_bend': hand_pos['pitch'],
            'vibrato': hand_pos['spread'] * 2,
            'hand': 'right' if is_right else 'left'
        }

def main():
    # Initialize
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(max_num_hands=2, model_complexity=1, min_detection_confidence=0.8)
    
    cap = cv2.VideoCapture(0)
    orchestra = GestureOrchestra()
    
    # Start audio thread
    audio_thread = threading.Thread(target=orchestra.synth.generate_audio, daemon=True)
    audio_thread.start()
    
    print("🎼 ADVANCED AIR GESTURE ORCHESTRA")
    print("Left Hand: Chords/Bass | Right Hand: Melody/Solo")
    print("Gestures: Fist=Stop, Thumb=Record/Prog, Palm=Chords, Fingers=Notes")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
            
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)
        
        active_voices = {}
        hand_data = []
        
        if results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                # Determine handedness (chiral test)
                wrist = hand_landmarks.landmark[0]
                is_right = wrist.x > 0.5  # Simple heuristic
                
                # Get gesture data
                fingers = orchestra.fingers_up(hand_landmarks.landmark)
                pos = orchestra.hand_position(hand_landmarks.landmark, frame.shape)
                gesture = orchestra.recognize_gesture(fingers, pos, is_right)
                
                hand_data.append(HandData(hand_landmarks.landmark, is_right, (0,0,0,0)))
                
                # Map to notes based on hand role
                if gesture['type'] == 'open_chord' and not is_right:
                    # Left hand chords
                    chord_notes = [60, 64, 67]  # C major base
                    for j, note in enumerate(chord_notes):
                        freq = 440 * (2 ** ((note + gesture['pitch_bend']) / 12))
                        active_voices[f"L{j}"] = (freq, freq * 0.25)  # FM params
                
                elif gesture['type'] == 'bass_note' and not is_right:
                    # Left hand bass
                    bass_freq = 130.81 + gesture['pitch_bend'] * 5  # C2 base
                    active_voices["bass"] = (bass_freq, bass_freq * 2)
                
                elif gesture['type'] == 'arpeggio' and is_right:
                    # Right hand melody
                    melody_notes = [67, 72, 76, 79]  # G major arpeggio
                    note_idx = int(pos['x'] * len(melody_notes)) % len(melody_notes)
                    freq = 440 * (2 ** ((melody_notes[note_idx] + gesture['pitch_bend']) / 12))
                    active_voices["melody"] = (freq, freq * 3)  # Higher mod for lead
                
                # Update synth
                orchestra.synth.voices = active_voices
                orchestra.synth.distortion = pos['spread'] * 0.5
        
        # Visual feedback
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (400, 200), (0,0,0), -1)
        cv2.putText(overlay, f"Voices: {len(active_voices)}", (15, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(overlay, f"Recording: {'●' if orchestra.synth.is_recording else '○'}", 
                   (15, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
        
        if hand_data:
            for hand in hand_data[:2]:
                mp_drawing.draw_landmarks(overlay, hand.landmarks, mp_hands.HAND_CONNECTIONS)
        
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        cv2.imshow('Advanced Air Gesture Orchestra 🎼', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'cv2'